<a href="https://colab.research.google.com/github/tonycccccc/tony.github.io/blob/main/Backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Original First Version Code
"""Fuse Logit and Attend operators

    Args:
      query_input: query output from query_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      key_input: key output from key_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      value_input: value output from value_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      bias_matrix: A tensor with shape [batch_size, 1, length_query, length_source],

    Symbol meanings:
      B : batch size
      T : key_length == length_qeury
      N : Head_num
      H : dim_per_head
      F : query_length == length_source (which also commented as )
    Returns:
      Attention layer output with shape [batch_size, length_query, hidden_size]

      logits = tf.einsum("BTNH,BFNH->BNFT", key, query)
      attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, value)
"""
# Randomly set granularity now
batch_granularity = 1
head_granularity = 1
element_granularity = 1

# Get shape of the matrix
batch_size, source_length, head_num, dim = query.shape.as_list()
_,key_length,_,_ = key.shape.as_list()
output = None
assert (head_granularity < head_num)

# The outermost loop loops through batch_size, with granularity as stride
for batch in range(0, batch_size, batch_granularity):
  flag = True
  suboutput = None
  # The inner loop loops through head, with granularity as stride
  for head in range(0, head_num, head_granularity):
    #Head_Dimension = len(tf.unstack(query, axis = 3))
    #for element in range(0, Head_Dimension, element_granularity):
      #for loop in range(1):
    output_from_unit_batch = np.zeros([batch_granularity, source_length, head_granularity, dim])
    # Check each bach inside each granularity
    for unit_batch in range(batch, batch + batch_granularity):
      output_from_unit_head = np.zeros([source_length, head_granularity, dim])
      # Check each head in the head granularity
      for unit_head in range(head, head + head_granularity):
        #for unit_element in range(element):
        logit_output = np.ones([source_length, key_length])
        for loop in range(2):
          if loop == 0:
            #iterate through query_length
            for unit_source in range(source_length):
              query_source = query_input[unit_batch, unit_source, unit_head, :]
              query_source = tf.reshape(query_source, [1, query_source.size]) # Tensor with 1 * 10 (unit_source_length * dim_per_head)
              key_source = key[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (key length * dim_per_head)
              #Result is F(1) * T matrix
              #Not sure if we need to do transpose here
              #result = tf.matmul(query_source, key_source) # Tensor with 1 * 64 size (unit_source_length * key_length)
              result = tf.matmul(query_source, tf.transpose(key_source)) # Tensor with 1 * 64 size (unit_source_length * key_length)
              result += bias[unit_batch, :, unit_source, :]
              #Row granularity
              result = tf.nn.softmax(result, name="attention_weights", axis=-1)
              logit_output[unit_source, :] = result
            #Logit output now is F * T matrix
            logit_output = tf.convert_to_tensor(logit_output)
          else:
            value_source = values[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (T * H) query_length * dim_per_head
            logit_output = tf.cast(logit_output, dtype=float)
            value_source = tf.cast(value_source, dtype=float)
            attention_output = tf.matmul(logit_output, value_source) #Matrix with size F * H
        output_from_unit_head[:, unit_head - head, :] = attention_output
      #Size now is F*N*H (output_from_unit_head)
      output_from_unit_batch[unit_batch - batch, :, :, :] = output_from_unit_head
    # Size now is BFNH (output from unit batch)
    if flag == True:
      suboutput = output_from_unit_batch
      flag = False
    else:
      suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
  if output == None:
    output = suboutput
  else:
    output = tf.concat([output, suboutput], 0)
print(output.shape)

In [ ]:
"""
Unit Test
"""
# Set a random query_input with size 32*32*2*10 Batch_size * source_length * num_head * dim_per_head
query_input = np.ones([32, 32, 2, 10])
temp = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
query_input= query_input * temp
query_input
# Create a random key_input with size 32 * 64 * 2 * 10 Batch_size * query_length * num_head * dim_per_head
key_input = np.ones([32, 64, 2, 10])
temp = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
key_input= key_input * temp
key_input
# Create random bias
bias = np.random.random([32, 1, 32, 64])
bias.shape
# Create random value matrix with size 32 * 64 * 2 * 10 Batch_size * length * head_num * dim_per_num
values = np.random.randint(0, 100, [32, 64, 2, 10])
values.shape

In [ ]:
# Code with separated test matrix
res = []
if (tf.config.list_physical_devices('GPU')):
  tf.config.experimental.reset_memory_stats('GPU:0')
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Before doing attention operations. Reset the memory")
  print(memory['peak'])
  print(memory['current'])
  for i in range(len(query_matrix)):
    query = query_matrix[i]
    key = key_matrix[i]
    bias = bias_matrix[i]
    value = value_matrix[i]
    logits = tf.einsum("BTNH,BFNH->BNFT", key, query)
    logits += bias
    weights = tf.nn.softmax(logits, name="attention_weights")
    weights = tf.nn.dropout(weights, rate=self.attention_dropout)
    old_attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, value)
    res.append(old_attention_output)
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Memory after attention operation.")
  print(memory['peak'])
  print(memory['current'])

In [ ]:
# Code with manually generated large matrix
if (tf.config.list_physical_devices('GPU')):
  tf.config.experimental.reset_memory_stats('GPU:0')
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Before doing attention operations. Reset the memory")
  print(memory['peak'])
  print(memory['current'])
  logits = tf.einsum("BTNH,BFNH->BNFT", key_matrix, query_matrix)
  #Manually set bias to 0.15
  logits += 0.15
  weights = tf.nn.softmax(logits, name="attention_weights")
  weights = tf.nn.dropout(weights, rate=self.attention_dropout)
  old_attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, value_matrix)
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Memory after attention operation.")
  print(memory['peak'])
  print(memory['current'])

In [ ]:
#@title First fusion function
# Fusion Method
import numpy as np
def fusion(query_input, key, bias, values):
    """Fuse Logit and Attend operators

    Args:
      query_input: query output from query_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      key_input: key output from key_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      value_input: value output from value_dense_layer with shape [batch_size, length, num_heads, dim_per_head].
      bias_matrix: A tensor with shape [batch_size, 1, length_query, length_source],

    Symbol meanings:
      B : batch size
      T : key_length == length_qeury
      N : Head_num
      H : dim_per_head
      F : query_length == length_source (which also commented as )
    Returns:
      Attention layer output with shape [batch_size, length_query, hidden_size]

      logits = tf.einsum("BTNH,BFNH->BNFT", key, query)
      attention_output = tf.einsum("BNFT,BTNH->BFNH", weights, value)
"""
  # Randomly set granularity now
    batch_granularity = 1
    head_granularity = 1
    element_granularity = 1

    # Get shape of the matrix
    batch_size, source_length, head_num, dim = query.shape.as_list()
    _,key_length,_,_ = key.shape.as_list()
    output = None
    assert (head_granularity < head_num)

    # The outermost loop loops through batch_size, with granularity as stride
    for batch in range(0, batch_size, batch_granularity):
      flag = True
      suboutput = None
      # The inner loop loops through head, with granularity as stride
      for head in range(0, head_num, head_granularity):
        #Head_Dimension = len(tf.unstack(query, axis = 3))
        #for element in range(0, Head_Dimension, element_granularity):
        #for loop in range(1):
        output_from_unit_batch = np.zeros([batch_granularity, source_length, head_granularity, dim])
        # Check each bach inside each granularity
        for unit_batch in range(batch, batch + batch_granularity):
          output_from_unit_head = np.zeros([source_length, head_granularity, dim])
          # Check each head in the head granularity
          for unit_head in range(head, head + head_granularity):
            #for unit_element in range(element):
            logit_output = np.ones([source_length, key_length])
            for loop in range(2):
              if loop == 0:
                #iterate through query_length
                for unit_source in range(source_length):
                  query_source = query_input[unit_batch, unit_source, unit_head, :]
                  query_source = tf.reshape(query_source, [1, query_source.size]) # Tensor with 1 * 10 (unit_source_length * dim_per_head)
                  key_source = key[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (key length * dim_per_head)
                  #Result is F(1) * T matrix
                  #Not sure if we need to do transpose here
                  #result = tf.matmul(query_source, key_source) # Tensor with 1 * 64 size (unit_source_length * key_length)
                  result = tf.matmul(query_source, tf.transpose(key_source)) # Tensor with 1 * 64 size (unit_source_length * key_length)
                  result += bias[unit_batch, :, unit_source, :]
                  #Row granularity
                  result = tf.nn.softmax(result, name="attention_weights", axis=-1)
                  logit_output[unit_source, :] = result
                #Logit output now is F * T matrix
                logit_output = tf.convert_to_tensor(logit_output)
              else:
                value_source = values[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (T * H) query_length * dim_per_head
                logit_output = tf.cast(logit_output, dtype=float)
                value_source = tf.cast(value_source, dtype=float)
                attention_output = tf.matmul(logit_output, value_source) #Matrix with size F * H
            output_from_unit_head[:, unit_head - head, :] = attention_output
          #Size now is F*N*H (output_from_unit_head)
          output_from_unit_batch[unit_batch - batch, :, :, :] = output_from_unit_head
        # Size now is BFNH (output from unit batch)
        if flag == True:
          suboutput = output_from_unit_batch
          flag = False
        else:
          suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
      if output == None:
        output = suboutput
      else:
        output = tf.concat([output, suboutput], 0)
    return output

In [ ]:
for i in range(0, 16, 8):
  print(i)

0
8


In [ ]:
import tensorflow as tf
a = tf.convert_to_tensor(np.array([[1, 0, 1, 0, 1], [0, 1, 0, 1, 0]]))
tf.gather(a, [1, 2, 3], axis = 1)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[0, 1, 0],
       [1, 0, 1]])>

In [ ]:
import numpy as np
new_res = []
if (tf.config.list_physical_devices('GPU')):
  tf.keras.backend.clear_session()
  tf.config.experimental.reset_memory_stats('GPU:0')
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Before doing attention operations. Reset the memory")
  print(memory['peak'])
  print(memory['current'])
  batch_size, source_length, head_num, dim = tf.shape(query).numpy()
  _,key_length,_,_, = tf.shape(key).numpy()
  batch_granularity = 32
  head_granularity = 8
  # Broadcasting to reshape the bias matrix to the shape as the logit BNFT
  bias = tf.reshape(bias, [batch_size, key_length, 1, -1])
  # The outermost loop loops through batch_size, with granularity as stride
  for batch in tf.range(0, batch_size, batch_granularity):
    for head in tf.range(0, head_num, head_granularity):
      batch_termination = batch + batch_granularity if batch + batch_granularity <= batch_size else batch_size
      for unit_batch in tf.range(batch, batch_termination):
        head_termination = head + head_granularity if head + head_granularity <= head_num else head_num
        for unit_head in tf.range(head, head + head_granularity):
          for loop in tf.range(2):
            if loop == 0:
                for query_source in tf.split(
                    query[unit_batch, :, unit_head, :], num_or_size_splits=1, axis=0):
                  key_source = key[unit_batch, :, unit_head, :]
                  result = tf.matmul(query_source, tf.transpose(key_source))
                  #result = bias[np.random.choice(bias.shape[0])] + result
                  #Row granularity
                  result = result + 0.15
                  result = tf.nn.softmax(result, name="attention_weights")
                  logit_output = result
            else:
              value_source = value[unit_batch, :, unit_head, :] 
              attention_output = tf.matmul(logit_output, value_source) #Matrix with size F * H
          if unit_head == head:
            attention_output = tf.expand_dims(attention_output, axis=1)
            output_from_unit_head = attention_output
          else:
           attention_output = tf.expand_dims(attention_output, axis=1)
           output_from_unit_head = tf.concat([output_from_unit_head, attention_output], 1)
       if unit_batch == batch:
         output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
         output_from_unit_batch = output_from_unit_head
         else:
           output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
           output_from_unit_batch = tf.concat([output_from_unit_batch, output_from_unit_head], 0)
       if head == 0:
         suboutput = output_from_unit_batch
      else:
       suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
   if batch == 0:
     output = suboutput
   else:
     output = tf.concat([output, suboutput], 0)
   new_res.append(output)
  memory = tf.config.experimental.get_memory_info('GPU:0')
print("Memory profiling using FLAT")
print(memory['peak'])
print(memory['current'])

In [ ]:
import numpy as np
new_res = []
if (tf.config.list_physical_devices('GPU')):
  tf.keras.backend.clear_session()
  tf.config.experimental.reset_memory_stats('GPU:0')
  memory = tf.config.experimental.get_memory_info('GPU:0')
  print("Before doing attention operations. Reset the memory")
  print(memory['peak'])
  print(memory['current'])

  # 将变量整合到function里面，尝试融合多个value矩阵concat形成的大矩阵， 尝试更高的granularity
  # logit_output = tf.Variable(0., shape=tf.TensorShape(None))
  # key_source = tf.Variable(0., shape=tf.TensorShape(None))
  # result = tf.Variable(0., shape=tf.TensorShape(None))
  # value_source = tf.Variable(0., shape=tf.TensorShape(None))
  # output = tf.Variable(0., shape=tf.TensorShape(None))
  # suboutput = tf.Variable(0., shape=tf.TensorShape(None))
  # output_from_unit_batch = tf.Variable(0., shape=tf.TensorShape(None))
  # output_from_unit_head = tf.Variable(0., shape=tf.TensorShape(None))
  for i in range(len(query_matrix)):
    query = query_matrix[i]
    key = key_matrix[i]
    bias = bias_matrix[i]
    value = value_matrix[i]
    batch_size, source_length, head_num, dim = tf.shape(query).numpy()
    _,key_length,_,_, = tf.shape(key).numpy()
    batch_granularity = 32
    head_granularity = 8
    # Broadcasting to reshape the bias matrix to the shape as the logit BNFT
    bias = tf.reshape(bias, [batch_size, key_length, 1, -1])
    # The outermost loop loops through batch_size, with granularity as stride
    for batch in tf.range(0, batch_size, batch_granularity):
      for head in tf.range(0, head_num, head_granularity):
        batch_termination = batch + batch_granularity if batch + batch_granularity <= batch_size else batch_size
        for unit_batch in tf.range(batch, batch_termination):
          head_termination = head + head_granularity if head + head_granularity <= head_num else head_num
          for unit_head in tf.range(head, head + head_granularity):
            for loop in tf.range(2):
              if loop == 0:
                for unit_source in tf.range(0, source_length):
                  # query_source = tf.Variable(0., shape=tf.TensorShape(None))
                  query_source = query[unit_batch, unit_source, unit_head, :]
                  query_source = tf.reshape(query_source, [1, -1]) # Tensor with 1 * 10 (unit_source_length * dim_per_head)
                  key_source = key[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (key length * dim_per_head)
                  result = tf.matmul(query_source, tf.transpose(key_source)) # Tensor with 1 * 64 size (unit_source_length * key_length)
                  result = bias[unit_batch, unit_source, :, :] + result
                  #Row granularity
                  result = tf.nn.softmax(result, name="attention_weights")
                  if unit_source == 0:
                    logit_output = result
                  else:
                    # Should be shape 45 * 45 here
                    logit_output = tf.concat([logit_output, result], axis=0)
              else:
                # attention_output = tf.Variable(0., shape=tf.TensorShape(None))
                value_source = value[unit_batch, :, unit_head, :] # Tensor with 64 * 10 (T * H) query_length * dim_per_head
                attention_output = tf.matmul(logit_output, value_source) #Matrix with size F * H
            if unit_head == head:
              attention_output = tf.expand_dims(attention_output, axis=1)
              output_from_unit_head = attention_output
            else:
              attention_output = tf.expand_dims(attention_output, axis=1)
              output_from_unit_head = tf.concat([output_from_unit_head, attention_output], 1)
          if unit_batch == batch:
            output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
            output_from_unit_batch = output_from_unit_head
          else:
            output_from_unit_head = tf.expand_dims(output_from_unit_head, axis=0)
            output_from_unit_batch = tf.concat([output_from_unit_batch, output_from_unit_head], 0)
        if head == 0:
          suboutput = output_from_unit_batch
        else:
          suboutput = tf.concat([suboutput, output_from_unit_batch], 2)
      if batch == 0:
        output = suboutput
      else:
        output = tf.concat([output, suboutput], 0)
    new_res.append(output)
    break 
  memory = tf.config.experimental.get_memory_info('GPU:0')
print("Memory profiling using FLAT")
print(memory['peak'])
print(memory['current'])

In [ ]:
import tensorflow as tf
import numpy as np
a = tf.convert_to_tensor(np.array([1, 2, 3]))
np.random.choice(a, [2, 2])

array([[1, 3],
       [3, 2]])